# Quickstart guide to querying the FANC dataset
FANC (*F*emale *A*dult *N*erve *C*ord, pronounced "fancy") is a 3d volume of the fly ventral nerve cord (VNC) imaged at synapse-level resolution.\
This guide is adapted from https://github.com/htem/FANC_auto_recon/blob/main/example_notebooks/FANC_Connectomics_General_Intro.ipynb to be more focused on ns&b 2023 neurons and less about 3d visualizations

This demo guide works on one computer, but if you want to set it up to use on your own see the following documentation:\
To explore the EM dataset, you need a caveclient token (see https://caveclient.readthedocs.io/en/latest/guide/authentication.html) \
To get verified as part of the FANC community see https://github.com/htem/FANC_auto_recon/wiki#collaborative-community \
You can explore just the 3d reconstructions of neurons without a token (see https://github.com/tuthill-lab/Lesser_Azevedo_2023/blob/main/plot_from_pkl.ipynb to get started)

In [ ]:
## import packages
import pandas as pd
import numpy as np
from matplotlib import pyplot,patches
import matplotlib.pyplot as plt
import seaborn as sns
import os

from caveclient import CAVEclient # EM specific
import json # for making links to neuroglancer views
import utils # EL functions to clean up the notebook

In [ ]:
# initialize access to the dataset - this step will be different when setting up on a new computer. see General Intro notebook above for info
client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)

## Annotation tables ##
To keep track of neurons, individuals upload "annotation tables" to label cells of interest to them 

In [ ]:
## call in annotation tables as dataframes
wingmotor_df = client.materialize.query_table('wing_motor_neuron_table_v0')
neckmotor_df = client.materialize.query_table('neck_motor_neuron_table_v0')
halteremotor_df = client.materialize.query_table('haltere_motor_neuron_table_v0')
legmotor_df = client.materialize.query_table('all_leg_motor_neuron_table_v0')
motor_df = pd.concat([wingmotor_df, neckmotor_df, halteremotor_df, legmotor_df]) # all motor dfs together 

premotor_df = client.materialize.query_table('wing_premotor_v4') # biased toward preMNs for the left side MNs
sensory_df = client.materialize.query_table('nerve_bundle_fibers_v0') # this table is not very well maintained
connective_df = client.materialize.query_table('neck_connective') # DN labels not included

### Columns of the annotation tables: ###
#### important columns ####
pt_root_id: unique identifier for the neuron (all the voxels that make up a segment)\
classification_system & cell_type: can be different for each table, they're two columns for extra identifiers determined by the table-creator\
for the wing motor neurons table, classification system specifies the nerve and the side and cell type specifies the muscle innervated

#### other columns ####
id: unique id of each entry within a table\
created: utc timestamp of when entry was added\
superceded_id: versioning thing\
pt_position: xyz coordinate of the point dropped for that segment
pt_supervoxel_id: unique identifier of the supervoxel at the xyz coordinate of the point dropped for that segment\

In [ ]:
## see the columns in a table
wingmotor_df.head() 


In [ ]:
##  to see info about table, like who created it and how complete it is

# tablename = 'neck_connective'
# client.materialize.get_table_metadata('tablename')

In [ ]:
# assign sign (excitatory or inhibitory) based on cell type
premotor_df = utils.NT_labels(premotor_df)

## Query synapses ## 
A table of all the predicted chemical synapses allows you to query upstream (presynaptic) or downstream (postsynaptic) of any cell.\
Using this script, you can identify the cells you want to query, whether to query upstream or downstream, and whether to constrain the connectivity to a particular cell type (e.g. only motor neurons downstream of the giant fiber)

In [ ]:
## choose a group of neurons to query their connectivity
## some possible neurons of interest - can add more
# ignore Boolean Series error for now...

DLMs = wingmotor_df.loc[wingmotor_df.cell_type.str.startswith('DLM')].pt_root_id.to_list() # DLMs 1 - 5
giant_fiber = [648518346489801267,648518346487752404]
desc_ant = [648518346488543822,648518346481626527]

## 19B: 20748 & 20754
ss19B = premotor_df[premotor_df.cell_type.isin(['19B'])].pt_root_id.to_list()
a19B, b19B = utils.subtypes_19B()

## 12B: 04728
ss04728 = premotor_df[premotor_df.cell_type.isin(['12B'])].pt_root_id.to_list()
## 12A: 48272
ss48272 = premotor_df[premotor_df.cell_type.isin(['12A'])][premotor_df.classification_system.isin(['local'])].pt_root_id.to_list() # remove primaries
## 3B/11B: 44002
ss44002 = [648518346477643278,648518346489463916,648518346488942057,648518346517121573,648518346494564151,648518346475387572,648518346483562133,648518346481006938,648518346470436606,648518346499869580,648518346489648435]

In [ ]:
query_neurons = ss44002
#query_neurons = [648518346475398272, 648518346489364825, 648518346465967408, 648518346482131604, 648518346497623542, 648518346484712605, 648518346526231767, 648518346514160427, 648518346494205042, 648518346520298833]
#query_neurons = a19B
#query_neurons = ss44002

## in the line below, change the term to "post_ids" or "pre_ids" depending on what you want to see
## post_ids to query upstream, pre_ids to query downstream
syn_df = client.materialize.synapse_query(pre_ids = query_neurons)


In [ ]:
## constrain synapse dataframe
# subset = wingmotor_df[premotor_df.sign.isin(['exc'])].pt_root_id.to_list()
# subset = ss19B
subset = motor_df.pt_root_id.to_list()

## in the line below, change the term to "post_pt_root_id" or "pre_pt_root_id" depending on what you want to see
## pre_pt_root_id to constrain upstream, post_pt_root_id to constrain downstream
syn_df_filtered = syn_df[syn_df.post_pt_root_id.isin(subset)]

In [ ]:
## see the top synaptic partners - constrained to neurons that are annotated

#### TOGGLE BETWEEN THE TWO LINES BELOW DEPENDING ON ABOVE QUERIES ####

## to see downstream partners:
labels_df = pd.DataFrame(syn_df_filtered.post_pt_root_id.value_counts().rename_axis('pt_root_id').reset_index(name='synapses')).merge(motor_df[['pt_root_id','classification_system','cell_type']],how = 'inner') 
# 
## to see upstream partners: 
#labels_df = pd.DataFrame(syn_df_filtered.post_pt_root_id.value_counts().rename_axis('pt_root_id').reset_index(name='synapses')).merge(premotor_df[['pt_root_id','classification_system','cell_type','sign']],how = 'inner') 


labels_df[0:10]

In [ ]:
# Copy and paste the Segment IDs in the "neuroglancer" tab in the box on the right next to the '+'

print('synaptic partners: {}'.format(labels_df.pt_root_id.to_list()[0:10]))

print('queried neurons: {}'.format(query_neurons))